In [1]:
import json
import typing
import os
from dataclasses import dataclass

# Test dataclasses

In [2]:
@dataclass(order=True)
class Name:
    name:str=None
    age: int=None

    def __str__(self):
        return f'{self.name}: {self.age} years'


@dataclass(order=True)
class TestClass:
    age: int =None
    name: str =None
    mail: str =None
    status: bool =None
    friends: typing.List[Name]=None

    def __str__(self) -> str:
        print(type(self.friends[0]))
        friend_text = "\n".join(str(f) for f in self.friends)                    
        return f'{self.name} is {self.age} and has {len(self.friends)} friends. These are: {friend_text}'

# Reading JSON

In [3]:
with open('utils/example_input.json') as content:
    json_dict = json.load(content)

# Marshaller

In [32]:
class Pyshaller():

    """
    TODO
    Add an unmarshall list function that reads every element in the list and creates the dataclass from it returning a list of the object
    Add the marshall function (That should be easier and recursive)
    """

    def __init__(self):
        self.simple_types = [
            'str',
            'bool',
            'float64',
            'int32',
            'int64',
            'datetime'            
        ]

    def is_simple(self, candidate: typing.Any) -> bool:
        for t in self.simple_types:
            if t in str(candidate) or candidate is None:
                return True
        return False

    def unmarshall(self, struct: object, json_dict: typing.Dict) -> object:
        # Dictionary of possible attributes of object
        attributes = [a for a in dir(struct) if not a.startswith('__')]
        for a in attributes:
            print(type(json_dict))
            if a in json_dict.keys():
                try:
                    value = json_dict[a]
                    setattr(struct, a, value)
                except ValueError:
                    print(f'There is no {a} key in the dictionary')
        return struct

    def unmarshall_list(self, struct: object, objects: typing.List[typing.Dict])-> typing.List[object]:    
        answer = []
        for item in objects:
            temp_struct = struct()
            answer.append(self.unmarshall(temp_struct, item))
        return answer

    def marshall(self, struct: object) -> typing.Dict:
        # TODO: Implement. Missing taking care of subclasses        
        answer = {}
        answer = self.add_attributes_to_dict(answer, struct)
        return answer
        
    def detect_type(self, struct: object) -> typing.Dict:
        attributes = [a for a in dir(struct) if not a.startswith('__')]
        for attr in attributes:
            value = getattr(struct, attr)
            attr_type = type(value)
            # Get special cases
            if getattr(struct, attr) is None:
                # It has a None value in the dataclass, it can be skipped for the JSON object.
                continue
            elif str(type(getattr(struct, attr))).__contains__('list'):
                print('list')
            elif not p.is_simple(attr_type):
                print(attr)
            target[attr] = getattr(struct, attr)
            


    def add_attributes_to_dict(self, target: typing.Dict, struct: object) -> typing.Dict:
        attributes = [a for a in dir(struct) if not a.startswith('__')]
        for attr in attributes:
            attr_type = type(getattr(struct, attr))
            if not self.is_simple(attr_type):
                # It is a class in itself
                # TODO Add the differentiation of lists and classes
                target[attr] = self.add_attributes_to_dict(target, attr)
            target[attr] = getattr(struct, attr)
        return target
        

In [5]:
json_dict

{'name': 'Jonas',
 'mail': 'jonas@gmail.com',
 'status': True,
 'friends': [{'name': 'juan', 'age': 20}, {'name': 'anna'}]}

In [6]:
p = Pyshaller()
t = TestClass()
t = p.unmarshall(t, json_dict)

<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>


In [9]:
t.friends, type(t.friends), type(t.friends[0])

([{'name': 'juan', 'age': 20}, {'name': 'anna'}], list, dict)

Note that every friend is currently a dictionary and not an instance of the class friend

In [10]:
t.friends = p.unmarshall_list(Name, t.friends)

<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>


In [11]:
type(t.friends[0])

__main__.Name

Now we have the instances. This process sadly has to be done manually using multiple lines of code in order to get the types in the nested classes

In [13]:
params = [a for a in dir(TestClass) if not a.startswith('__')]
params

['age', 'friends', 'mail', 'name', 'status']

In [16]:
type(getattr(TestClass, 'age'))

NoneType

I cannot get the intended type of the class. Only its current value.

In [34]:
p.is_simple(t.age)

False

In [31]:
for attr in attributes:
    attr_type = type(getattr(struct, attr))
    print(f'ATT: {attr} TYPE: {attr_type}')
    if getattr(struct, attr) is None:
        continue
    elif str(type(getattr(struct, attr))).__contains__('list'):
        print('list')
    elif not p.is_simple(attr_type):
        print(attr)
    target[attr] = getattr(struct, attr)

ATT: age TYPE: <class 'NoneType'>
ATT: friends TYPE: <class 'list'>
list
ATT: mail TYPE: <class 'str'>
ATT: name TYPE: <class 'str'>
ATT: status TYPE: <class 'bool'>


In [30]:
type(t.friends).__contains__('list')

TypeError: descriptor '__contains__' requires a 'list' object but received a 'str'